In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential
from keras import metrics
from keras.models import model_from_json

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from PIL import Image


/home/doxanh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/doxanh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/doxanh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/doxanh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [2]:
print(keras.__version__)

2.3.1


In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
print(tf.__version__)
print(device_lib.list_local_devices())

In [ ]:
#VGG-Face model
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
model.load_weights('model_checkpoint/vgg_face_weights.h5')

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

# Train and test age predictions

In [ ]:
train_it = datagen.flow_from_directory('age_dataset/train', target_size=(224, 224), class_mode='categorical', batch_size=8)
val_it = datagen.flow_from_directory('age_dataset/val', target_size=(224, 224), class_mode='categorical', batch_size=8)
test_it = datagen.flow_from_directory('age_dataset/test', target_size=(224, 224), class_mode='categorical', batch_size=8)

In [ ]:
classes = 101
for layer in model.layers[:-7]:
    layer.trainable = False

base_model_output = Sequential()
base_model_output = Convolution2D(classes, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = Flatten()(base_model_output)
base_model_output = Activation('softmax')(base_model_output)

age_model = Model(inputs=model.input, outputs=base_model_output)

In [ ]:
sgd = keras.optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

age_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=1e-4, decay=1e-6),
                  metrics=['accuracy']
                 )

age_model.summary()

In [ ]:
checkpointer = ModelCheckpoint(
    filepath='model_checkpoint/classifier_age_model_weights.h5',
    monitor = "val_loss",
    verbose=1,
    save_best_only=True,
    mode = 'auto'
)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

callbacks = [checkpointer, es]

In [ ]:
age_model.load_weights('model_checkpoint/age_model_weights.h5')

In [ ]:
hitory = age_model.fit_generator(train_it, epochs=250, verbose=1, callbacks=callbacks, validation_data=val_it)

In [ ]:
age_model.evaluate_generator(test_it, verbose=1)

In [ ]:
test_y = []
test_x = []
for i in range(len(test_it)):
    batch_x, batch_y = test_it.next()
    for j in range(batch_x.shape[0]):
        test_x.append(batch_x[j])
        test_y.append(np.argmax(batch_y[j]))
        
test_y = np.array(test_y)
test_x = np.array(test_x)
test_x = test_x.reshape(test_x.shape[0], 224, 224, 3)
test_x /= 255

In [ ]:
test_x.shape, test_y.shape

In [ ]:
predictions = age_model.predict(test_x)

In [ ]:
output_indexes = np.array([i for i in range(0, 101)])
apparent_predictions = np.sum(predictions*output_indexes, axis=1)

In [ ]:
apparent_predictions = []
for i in range(predictions.shape[0]):
    apparent_predictions.append(np.argmax(predictions[i]))

apparent_predictions = np.array(apparent_predictions)

In [ ]:
mae = 0

for i in range(0 ,apparent_predictions.shape[0]):
    prediction = int(apparent_predictions[i])
    abs_error = abs(prediction - test_y[i])
    mae = mae + abs_error
    
mae = mae / apparent_predictions.shape[0]

print("mae: ",mae)
print("instances: ",apparent_predictions.shape[0])

# Train and test gender predictions

In [ ]:
gender_train_it = datagen.flow_from_directory('gender_dataset/train', target_size=(224, 224), class_mode='categorical', batch_size=16)
gender_val_it = datagen.flow_from_directory('gender_dataset/val', target_size=(224, 224), class_mode='categorical', batch_size=16)
gender_test_it = datagen.flow_from_directory('gender_dataset/test', target_size=(224, 224), class_mode='categorical', batch_size=16)

In [ ]:
classes = 2

for layer in model.layers[:-7]:
    layer.trainable = False

base_model_output = Sequential()
base_model_output = Convolution2D(classes, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = Flatten()(base_model_output)
base_model_output = Activation('softmax')(base_model_output)

gender_model = Model(inputs=model.input, outputs=base_model_output)

In [ ]:
gender_model.compile(loss='categorical_crossentropy',
                     optimizer=keras.optimizers.Adam(lr=1e-4, decay=1e-6), 
                     metrics=['accuracy'])

In [ ]:
gender_checkpointer = ModelCheckpoint(
    filepath='model_checkpoint/gender_model_weights.h5',
    monitor = "val_loss",
    verbose=1,
    save_best_only=True,
    mode = 'auto'
)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

gender_callbacks = [gender_checkpointer, es]

In [ ]:
gender_model.load_weights('model_checkpoint/gender_model_weights.h5')

In [ ]:
hitory = gender_model.fit_generator(gender_train_it, epochs=250, verbose=1, callbacks=gender_callbacks, validation_data=gender_val_it)

In [ ]:
test_y = []
test_x = []
for i in range(len(gender_test_it)):
    batch_x, batch_y = gender_test_it.next()
    for j in range(batch_x.shape[0]):
        test_x.append(batch_x[j])
        test_y.append(batch_y[j])
        
test_y = np.array(test_y)
test_x = np.array(test_x)
test_x = test_x.reshape(test_x.shape[0], 224, 224, 3)
# test_x /= 255

In [ ]:
# gender_model.evaluate(test_x, test_y, verbose=1)
gender_model.evaluate_generator(gender_test_it, verbose=1)

In [ ]:
gender_predictions = gender_model.predict(test_x, verbose=1)

In [ ]:
gender_pred_list = []
gender_actual_list = []

for i in gender_predictions:
    gender_pred_list.append(np.argmax(i))
    
for i in test_y: 
    gender_actual_list.append(np.argmax(i))
    
confusion_matrix = confusion_matrix(gender_actual_list, gender_pred_list)

In [ ]:
confusion_matrix